In [1]:
debug = True

import pandas as pd  
import os
import plotly as py
import plotly.graph_objects as go
import ipywidgets as widgets
import numpy as np
from scipy import special
import time

# offline mode for charts, so it stays local
py.offline.init_notebook_mode(connected=True)

# read in curve data'
dirCWD    = os.getcwd()
dirIntermediate = os.path.join(dirCWD,'_Friction_Factors')
dirResults = os.path.join(dirCWD,'_Friction_Factors')

binsize = 2

observedStage = 'v9 Final Smoothing'

baseTLFLoc = r'A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9.0Beta\WF TDM v9.0 - 2022-11-01\Scenarios\BY_2019\3_Distribute\TLF'

# some trip purposes are added together for information purposes
dfTripPurpSubtotals = pd.DataFrame([
    ['HBOth','HBO' ],
    ['HBShp','HBO' ],
    ['NHBW' ,'NHB' ],
    ['NHBNW','NHB' ],
    ['IX'   ,'IXXI'],
    ['XI'   ,'IXXI']
],columns=(['TRIPPURP','TRIPPURP_SUB']))
if debug: display(dfTripPurpSubtotals)

dfCalibrationRounds = pd.DataFrame([
    ['0-Initial from v832',r'A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9.0Beta\WF TDM v9.0 - 2022-11-01\1_Inputs\0_GlobalData\3_Distribute\FricFactor_AllPurp.csv', False, False],
    ['1-Second from v832',r'A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9.0Beta\WF TDM v9.0 - 2022-11-01\1_Inputs\0_GlobalData\3_Distribute\FricFactor_AllPurp.csv', False, False]
],columns=(['FF_CALIB_ROUND','FILENAMEFF','LOCKFF','LOCKTLF']))
if debug: display(dfCalibrationRounds)

,TRIPPURP,TRIPPURP_SUB
0,HBOth,HBO
1,HBShp,HBO
2,NHBW,NHB
3,NHBNW,NHB
4,IX,IXXI
5,XI,IXXI


,FF_CALIB_ROUND,FILENAMEFF,LOCKFF,LOCKTLF
0,0-Initial from v832,A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9...,False,False
1,1-Second from v832,A:\1 - TDM\3 - Model Dev\1 - WF\2 - Sandbox\v9...,False,False


# Observed Trip Length Frequencies

In [2]:
# read in observed TLF data to be used
dfTLF_Obs = pd.read_csv(r'A:\1 - TDM\2 - Estimate Param\_General Parameters\7 - Update Obs TLF\3 - Create Obs TLFs for all Trips\results\dfTLF_Obs_wSmoothed_20221130-133903.csv')

#filter by the STAGE for 
dfTLF_Obs = dfTLF_Obs[dfTLF_Obs['STAGE']==observedStage].copy()
dfTLF_Obs = dfTLF_Obs.drop(columns=('STAGE'))
if debug: display(dfTLF_Obs)

,BIN,TRIPPURP,FREQ,TLFTYPE
41726,0.0,HBW,0.151770,Dist
41727,2.0,HBW,0.148953,Dist
41728,4.0,HBW,0.128122,Dist
41729,6.0,HBW,0.106026,Dist
41730,8.0,HBW,0.090793,Dist
...,...,...,...,...
47871,402.0,XI_HV,0.000000,GC
47872,404.0,XI_HV,0.000000,GC
47873,406.0,XI_HV,0.000000,GC
47874,408.0,XI_HV,0.000000,GC


In [3]:
# show max bin size for TLF type to use in updating TDM TLF code
if debug: display(dfTLF_Obs[dfTLF_Obs['FREQ']>0].groupby(['TLFTYPE'],as_index=False).agg(MAXBIN=('BIN','max')))

,TLFTYPE,MAXBIN
0,Dist,160.0
1,GC,392.0
2,Time,182.0


In [4]:
# Check TLF Observed. FREQ should all sum to 1! Also show average trip length
dfTLF_Obs['BINxFREQ'] = dfTLF_Obs['BIN'] * dfTLF_Obs['FREQ']
dfTLF_Obs_Stats = dfTLF_Obs.groupby(['TRIPPURP','TLFTYPE'],as_index=False).agg(FREQ_SUM=('FREQ','sum'),AVG_TRIP_LEN=('BINxFREQ','sum'))
dfTLF_Obs = dfTLF_Obs.drop(columns=('BINxFREQ'))
if debug: display(dfTLF_Obs_Stats)

,TRIPPURP,TLFTYPE,FREQ_SUM,AVG_TRIP_LEN
0,HBOth,Dist,1.0,3.665819
1,HBOth,GC,1.0,16.186377
2,HBOth,Time,1.0,10.175627
3,HBSchPr,Dist,1.0,0.622159
4,HBSchPr,GC,1.0,8.149783
5,HBSchPr,Time,1.0,5.534968
6,HBSchSc,Dist,1.0,1.975023
7,HBSchSc,GC,1.0,11.332930
8,HBSchSc,Time,1.0,7.526669
9,HBShp,Dist,1.0,2.762367


In [5]:
import plotly as py
import plotly.graph_objects as go
import ipywidgets as widgets
import numpy as np
from scipy import special

py.offline.init_notebook_mode(connected=True)

# Observed Plots

In [6]:
#PLOTTING FUNCTION

# chart preset zoom extents
dfZoomPresets = pd.DataFrame([
    ['Begin' ,     0,   100,     0,np.NaN],
    ['Mid'   ,    30,   160,     0, 0.018],
    ['Tail'  ,   100,   400,     0, 0.001],
    ['All'   ,np.NaN,np.NaN,np.NaN,np.NaN],
    ['Custom',np.NaN,np.NaN,np.NaN,np.NaN],
], columns=('PRESET','XMIN','XMAX','YMIN','YMAX'))
dfZoomPresets

def update_plot_obs(trippurps, tlftypes, zoompreset, xmin, xmax, ymin, ymax):

    dfAvgTripLen = pd.DataFrame()
    
    data = []
    for trippurp in trippurps:
        for tlftype in tlftypes:

            # data for plotting from filtered dataframe
            plotdata = dfTLF_Obs[(dfTLF_Obs['TRIPPURP']==trippurp) & (dfTLF_Obs['TLFTYPE']==tlftype)]

            # fill any NaN values with zeros
            plotdata = plotdata.fillna(0)
            
            xplot = plotdata['BIN']
            yplot = plotdata['FREQ']
            name  = trippurp + ' ' + tlftype 

            trace1 = go.Scatter(
                x=xplot,
                y=yplot,
                mode='lines',
                name=name,
                line=dict(
                    shape='spline'
                )
            )
            data.append(trace1)
                
    # get zoom presets
    if (zoompreset=='Custom'):
        _xmin = xmin
        _xmax = xmax
        _ymin = ymin
        _ymax = ymax
    else:
        _xmin = dfZoomPresets[dfZoomPresets['PRESET']==zoompreset]['XMIN'].values[0]
        _xmax = dfZoomPresets[dfZoomPresets['PRESET']==zoompreset]['XMAX'].values[0]
        _ymin = dfZoomPresets[dfZoomPresets['PRESET']==zoompreset]['YMIN'].values[0]
        _ymax = dfZoomPresets[dfZoomPresets['PRESET']==zoompreset]['YMAX'].values[0]

    layout = go.Layout(
        title='Trip Length Frequencies - Observed Data',
        yaxis=dict(
            title='',
            range=(_ymin,_ymax)
        ),
        xaxis=dict(
            title='GC/Dist/Time',
            range=(_xmin,_xmax)
        ),
        #width=1600,
        height=450
    )
    
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)

In [7]:
# MAKE INTERACTIVE CHART
py.offline.init_notebook_mode(connected=True)

lstTripPurp  = dfTLF_Obs['TRIPPURP'].unique().tolist() # HBW, HBShp, HBOth,...
lstTLFType   = dfTLF_Obs['TLFTYPE' ].unique().tolist() # GC, Time, Dist
    
selectTripPurp  = widgets.SelectMultiple(options=lstTripPurp, value=('HBW'        ,), description='Trip Purpose')
selectTLFType   = widgets.SelectMultiple(options=lstTLFType , value=('GC'         ,), description='TLF Type'    )
selectPreset    = widgets.Select(options=dfZoomPresets['PRESET'].tolist(), value='All', description='Zoom Presets')

#custom extents
xmin = widgets.Text(value='0', description='X Min')
xmax = widgets.Text(value='60', description='X Max')
ymin = widgets.Text(value='0', description='Y Min')
ymax = widgets.Text(value='.01', description='Y Max')

widgets.interactive(update_plot_obs, trippurps=selectTripPurp, tlftypes=selectTLFType, zoompreset=selectPreset, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)

interactive(children=(SelectMultiple(description='Trip Purpose', index=(0,), options=('HBW', 'HBShp', 'HBOth',…

# Friction Factors

In [8]:
# read Friction Factors into array ordered by iteration

# Friction Factors array that stores a separate dataframe for each Calibration Round
dfsFF=[]

for index, row in dfCalibrationRounds.iterrows():
    filename = os.path.join(dirIntermediate, row['FF_CALIB_ROUND'] + ' - friction factors.csv')
    if row['LOCKFF']==False:
        display ('Reading friction factors for ' + row['FF_CALIB_ROUND'])
        dfFF = pd.read_csv(row['FILENAMEFF'])

        # make sure id column is always same: BIN
        dfFF = dfFF.rename(columns={';MINUTE':'BIN','Mile':'BIN','Mil':'BIN','Min':'BIN','Bin':'BIN'})

        # melt all columns but first
        dfFF = pd.melt(dfFF, id_vars=['BIN'], value_vars=dfFF.columns[1:].tolist(), ignore_index=False, var_name='TRIPPURP', value_name='FF')
        
        dfFF.to_csv(filename, index=False)

    else:
        dfFF = pd.read_csv(filename)

    if debug: display(dfFF)

    # append to array of Friction Factors
    dfsFF.append(dfFF)



'Reading friction factors for 0-Initial from v832'

,BIN,TRIPPURP,FF
0,0,HBW,1.000000
1,2,HBW,0.999800
2,4,HBW,0.609169
3,6,HBW,0.190323
4,8,HBW,0.098000
...,...,...,...
146,292,XI_HV,0.000000
147,294,XI_HV,0.000000
148,296,XI_HV,0.000000
149,298,XI_HV,0.000000


'Reading friction factors for 1-Second from v832'

,BIN,TRIPPURP,FF
0,0,HBW,1.000000
1,2,HBW,0.999800
2,4,HBW,0.609169
3,6,HBW,0.190323
4,8,HBW,0.098000
...,...,...,...
146,292,XI_HV,0.000000
147,294,XI_HV,0.000000
148,296,XI_HV,0.000000
149,298,XI_HV,0.000000


# Read in TDM Trip Length Frequencies

In [9]:
# read in trips from TDM sorted into TLF bins

# array of dataframes to hold TLFs from each round of calibration
dfsTLF_TDM = []

for index, row in dfCalibrationRounds.iterrows():

    display('Reading ' + row['FF_CALIB_ROUND'] + ' TLFs')

    filename = os.path.join(dirIntermediate, row['FF_CALIB_ROUND'] + ' - tlfs.csv')

    if row['LOCKTLF']==False:

        tlfs=['Cost','Dist','Time']

        dfTLFTDMTrips = pd.DataFrame()

        for tlf in tlfs:
            
            # read in csv for tlf
            dfRead = pd.read_csv(os.path.join(baseTLFLoc,'TLF_' + tlf + '.csv'))
            
            # make sure id column is always same: BIN
            dfRead = dfRead.rename(columns={';BIN':'BIN',';MINUTE':'BIN','Mile':'BIN','Mil':'BIN','Min':'BIN','Bin':'BIN'})

            dfRead = pd.melt(dfRead, id_vars=['BIN'], value_vars=dfRead.columns[1:].tolist(), ignore_index=False, var_name='TRIPPURP', value_name='TRIPS')

            # set TLF value, rename Cost to GC
            if tlf=='Cost':
                dfRead['TLFTYPE'] = 'GC'
            else:
                dfRead['TLFTYPE'] = tlf

            # reorder columns
            dfRead = dfRead[['TLFTYPE','TRIPPURP','BIN','TRIPS']]

            # concat data into single dataframe
            dfTLFTDMTrips = pd.concat([dfTLFTDMTrips, dfRead], ignore_index=True)

        if debug: display(dfTLFTDMTrips)

        # create subtotals
        dfTLFTDMTripsForSubtotals = pd.DataFrame.merge(dfTLFTDMTrips, dfTripPurpSubtotals, on=('TRIPPURP'))
        dfTLFTDMTripsForSubtotals = dfTLFTDMTripsForSubtotals.groupby(['TLFTYPE','TRIPPURP_SUB','BIN'],as_index=False).agg(TRIPS=('TRIPS','sum'),COUNT=('TRIPS','size'))
        dfTLFTDMTripsForSubtotals = dfTLFTDMTripsForSubtotals[['TLFTYPE','TRIPPURP_SUB','BIN','TRIPS']]
        dfTLFTDMTripsForSubtotals = dfTLFTDMTripsForSubtotals.rename(columns={'TRIPPURP_SUB':'TRIPPURP'})
        if debug: display (dfTLFTDMTripsForSubtotals)

        # concat subtotals
        dfTLFTDMTrips = pd.concat([dfTLFTDMTrips,dfTLFTDMTripsForSubtotals], ignore_index=True)
        if debug: display(dfTLFTDMTrips)

        # calculated collapsed bins
        from math import floor
        def round_to_binsize(x):
            return int(binsize * floor(float(x)/binsize))
        dfTLFTDMTrips['BIN_COLLAPSE'] = dfTLFTDMTrips['BIN'].apply(lambda x: round_to_binsize(x))
        if debug: display(dfTLFTDMTrips)

        # aggregate to collapsed bins
        dfTLFTDMTrips = dfTLFTDMTrips.groupby(['TLFTYPE','TRIPPURP','BIN_COLLAPSE'], as_index=False).agg(TRIPS=('TRIPS','sum'))
        dfTLFTDMTrips = dfTLFTDMTrips.rename(columns={'BIN_COLLAPSE':'BIN'})
        if debug: display(dfTLFTDMTrips)

        # calculate percent distribution
        dfTLFTDMTripTotals = dfTLFTDMTrips.groupby(['TLFTYPE','TRIPPURP'], as_index=False).agg(TRIP_TOTAL=('TRIPS','sum'))
        if debug: display(dfTLFTDMTripTotals)

        # join to toals
        dfTLFTDMTripDist = pd.DataFrame.merge(dfTLFTDMTrips, dfTLFTDMTripTotals, on=('TLFTYPE','TRIPPURP'))
        dfTLFTDMTripDist['FREQ'] = dfTLFTDMTripDist['TRIPS'] / dfTLFTDMTripDist['TRIP_TOTAL']
        display(dfTLFTDMTripDist)

        dfTLF_TDM = dfTLFTDMTripDist[['TLFTYPE','TRIPPURP','BIN','FREQ']]
        display(dfTLF_TDM)

        # check to see if add up to 1.0
        dfCheck = dfTLF_TDM.groupby(['TLFTYPE','TRIPPURP'], as_index=False).agg(FREQ_SUM=('FREQ','sum'))
        # only display results out of range
        display('Not adding to 1 (if empty dataset, YAY!!!):')
        display(dfCheck[(dfCheck['FREQ_SUM']<0.9999999) | (dfCheck['FREQ_SUM']>1.0000001)])

        dfTLF_TDM.to_csv(filename, index=False)

    else:
        dfTLF_TDM = pd.read_csv(filename)

    #add calibration round back into table
    dfTLF_TDM['FF_CALIB_ROUND'] = row['FF_CALIB_ROUND']
    # append dataframe to array
    dfsTLF_TDM.append(dfTLF_TDM)


'Reading 0-Initial from v832 TLFs'

,TLFTYPE,TRIPPURP,BIN,TRIPS
0,GC,HBW,0,0.00
1,GC,HBW,1,0.00
2,GC,HBW,2,5806.63
3,GC,HBW,3,21119.70
4,GC,HBW,4,16761.40
...,...,...,...,...
12649,Time,Tel_NHBW,196,0.00
12650,Time,Tel_NHBW,197,0.00
12651,Time,Tel_NHBW,198,0.00
12652,Time,Tel_NHBW,199,0.00


,TLFTYPE,TRIPPURP,BIN,TRIPS
0,Dist,HBO,0,910649.34
1,Dist,HBO,1,1034931.17
2,Dist,HBO,2,674422.97
3,Dist,HBO,3,425223.32
4,Dist,HBO,4,282306.66
...,...,...,...,...
2104,Time,NHB,196,0.00
2105,Time,NHB,197,0.00
2106,Time,NHB,198,0.00
2107,Time,NHB,199,0.00


,TLFTYPE,TRIPPURP,BIN,TRIPS
0,GC,HBW,0,0.00
1,GC,HBW,1,0.00
2,GC,HBW,2,5806.63
3,GC,HBW,3,21119.70
4,GC,HBW,4,16761.40
...,...,...,...,...
14758,Time,NHB,196,0.00
14759,Time,NHB,197,0.00
14760,Time,NHB,198,0.00
14761,Time,NHB,199,0.00


,TLFTYPE,TRIPPURP,BIN,TRIPS,BIN_COLLAPSE
0,GC,HBW,0,0.00,0
1,GC,HBW,1,0.00,0
2,GC,HBW,2,5806.63,2
3,GC,HBW,3,21119.70,2
4,GC,HBW,4,16761.40,4
...,...,...,...,...,...
14758,Time,NHB,196,0.00,196
14759,Time,NHB,197,0.00,196
14760,Time,NHB,198,0.00,198
14761,Time,NHB,199,0.00,198


...

,TLFTYPE,TRIPPURP,BIN,TRIPS,TRIP_TOTAL,FREQ
0,Dist,Ext_HV,0,17.60,31844.37,5.526880e-04
1,Dist,Ext_HV,2,144.10,31844.37,4.525133e-03
2,Dist,Ext_HV,4,399.32,31844.37,1.253974e-02
3,Dist,Ext_HV,6,807.51,31844.37,2.535801e-02
4,Dist,Ext_HV,8,892.52,31844.37,2.802756e-02
...,...,...,...,...,...,...
7408,Time,XX,192,2.18,15337.83,1.421322e-04
7409,Time,XX,194,0.49,15337.83,3.194715e-05
7410,Time,XX,196,0.01,15337.83,6.519827e-07
7411,Time,XX,198,0.20,15337.83,1.303965e-05


,TLFTYPE,TRIPPURP,BIN,FREQ
0,Dist,Ext_HV,0,5.526880e-04
1,Dist,Ext_HV,2,4.525133e-03
2,Dist,Ext_HV,4,1.253974e-02
3,Dist,Ext_HV,6,2.535801e-02
4,Dist,Ext_HV,8,2.802756e-02
...,...,...,...,...
7408,Time,XX,192,1.421322e-04
7409,Time,XX,194,3.194715e-05
7410,Time,XX,196,6.519827e-07
7411,Time,XX,198,1.303965e-05


'Not adding to 1 (if empty dataset, YAY!!!):'

,TLFTYPE,TRIPPURP,FREQ_SUM


C:\Users\cday\AppData\Local\ESRI\conda\envs\arcgispro-py3-plotly\lib\site-packages\ipykernel_launcher.py:89: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Model vs Observed Plots

Compare Average Trip Length Between Modeled and Observed Data

In [18]:
dfTLF_Obs_N = dfTLF_Obs
dfTLF_Obs_N['FF_CALIB_ROUND'] = 'Observed'

#Merge together all Modeled data and Observed data
dfObsModelMerge = (pd.concat(dfsTLF_TDM).append(dfTLF_Obs_N))   

#CALCUATE TRIP LENGTH AVERAGES AS TABLE
dfObsModelMerge['BINxFREQ'] = dfObsModelMerge['BIN'] * dfObsModelMerge['FREQ']
dfObsModelMerge_Stats = (dfObsModelMerge
    .groupby(['FF_CALIB_ROUND','TRIPPURP','TLFTYPE'],as_index=False)
        .agg(FREQ_SUM=('FREQ','sum'),AVG_TRIP_LEN=('BINxFREQ','sum')))

if debug: display(dfObsModelMerge_Stats)

,FF_CALIB_ROUND,TRIPPURP,TLFTYPE,FREQ_SUM,AVG_TRIP_LEN
0,0-Initial from v832,Ext_HV,Dist,1.0,39.391920
1,0-Initial from v832,Ext_HV,GC,1.0,94.845462
2,0-Initial from v832,Ext_HV,Time,1.0,43.453935
3,0-Initial from v832,Ext_MD,Dist,1.0,31.847548
4,0-Initial from v832,Ext_MD,GC,1.0,74.702942
...,...,...,...,...,...
166,Observed,XI_HV,GC,1.0,162.170483
167,Observed,XI_HV,Time,1.0,72.560994
168,Observed,XI_MD,Dist,1.0,42.125143
169,Observed,XI_MD,GC,1.0,99.290194


In [13]:
dfTLF_MvO_Wide = (dfObsModelMerge_Stats
    .pivot(index=['TRIPPURP','TLFTYPE'],columns="FF_CALIB_ROUND",values="AVG_TRIP_LEN")
    .reset_index(['TRIPPURP','TLFTYPE'])
    .assign(DIF = lambda x: (x['Observed'] - x['0-Initial from v832'])/(x['0-Initial from v832'])*100))
display(dfTLF_MvO_Wide)

FF_CALIB_ROUND,TRIPPURP,TLFTYPE,0-Initial from v832,1-Second from v832,Observed,DIF
0,Ext_HV,Dist,39.391920,39.391920,NaN,NaN
1,Ext_HV,GC,94.845462,94.845462,NaN,NaN
2,Ext_HV,Time,43.453935,43.453935,NaN,NaN
3,Ext_MD,Dist,31.847548,31.847548,NaN,NaN
4,Ext_MD,GC,74.702942,74.702942,NaN,NaN
...,...,...,...,...,...,...
82,XI_MD,GC,NaN,NaN,99.290194,NaN
83,XI_MD,Time,NaN,NaN,48.007368,NaN
84,XX,Dist,55.216545,55.216545,NaN,NaN
85,XX,GC,117.314474,117.314474,NaN,NaN


In [79]:
import plotly.graph_objects as go
import plotly.express as px

def update_df_mod_vs_obs(modeloption,tlfoption,trippurpoption):
    modellist = list(modeloption)
    modellist.append('Observed')

    dfObsModelMerge_Stats2 =(dfObsModelMerge_Stats
        .loc[dfObsModelMerge_Stats['FF_CALIB_ROUND'].isin(modellist)])
    dfObsModelMerge_Stats3 = (dfObsModelMerge_Stats2
        .loc[dfObsModelMerge_Stats2['TLFTYPE'].isin(tlfoption)])
    dfObsModelMerge_Stats4 = (dfObsModelMerge_Stats3
        .loc[dfObsModelMerge_Stats3['TRIPPURP'].isin(trippurpoption)])
   
    if len(modellist) > 2:
        dfTLF_MvO_Wide2 = (dfObsModelMerge_Stats4
            .pivot(index=['TRIPPURP','TLFTYPE'],columns="FF_CALIB_ROUND",values="AVG_TRIP_LEN")
            .reset_index(['TRIPPURP','TLFTYPE']))
    else:
        dfTLF_MvO_Wide2 = (dfObsModelMerge_Stats4
            .pivot(index=['TRIPPURP','TLFTYPE'],columns="FF_CALIB_ROUND",values="AVG_TRIP_LEN")
            .reset_index(['TRIPPURP','TLFTYPE'])
            .assign(PerChange = lambda x: (x['Observed'] - x[modellist[0]])/(x[modellist[0]])*100))

    fig = go.Figure(data=[go.Table(
                    header = dict(values=list(dfTLF_MvO_Wide2.columns),
                                  fill_color='paleturquoise',
                                  align='left'),
                    cells =  dict(values=dfTLF_MvO_Wide2.transpose().values.tolist(),
                                 fill_color='lavender',
                                 align='left'))
                    ])

    figbar2 = px.bar(dfObsModelMerge_Stats4, 
                     x = 'TRIPPURP', 
                     y = 'AVG_TRIP_LEN', 
                     color = 'FF_CALIB_ROUND', 
                     barmode = 'group'
                    )

    figbar2.show()
    py.offline.iplot(fig)  


In [80]:
py.offline.init_notebook_mode(connected=True)

lstModOpt  = dfObsModelMerge['FF_CALIB_ROUND'].unique().tolist()
lstModOptShort = [round for round in lstModOpt if round != 'Observed']  
lstTLFType   = dfObsModelMerge['TLFTYPE' ].unique().tolist() # GC, Time, Dist
lstTripPurp  = dfTLF_Obs['TRIPPURP'].unique().tolist() # HBW, HBShp, HBOth,...
    
selectModelOption  = widgets.SelectMultiple(options=lstModOptShort     , value=('0-Initial from v832',), description='Model Run')
selectTLFOption    = widgets.SelectMultiple(options=lstTLFType , value=('GC' ,)               , description='TLF Type')
selectTripPurp     = widgets.SelectMultiple(options=lstTripPurp, value=('HBW',)               ,description='Trip Purpose')

widgets.interactive(update_df_mod_vs_obs,modeloption=selectModelOption, tlfoption=selectTLFOption, trippurpoption=selectTripPurp)

interactive(children=(SelectMultiple(description='Model Run', index=(0,), options=('0-Initial from v832', '1-S…

Comparing Trip Length Frequency Graphs between Modeled and Observed Data

In [22]:
#PLOTTING FUNCTION

# chart preset zoom extents
dfZoomPresets = pd.DataFrame([
    ['Begin' ,     0,   100,     0,np.NaN],
    ['Mid'   ,    30,   160,     0, 0.018],
    ['Tail'  ,   100,   400,     0, 0.001],
    ['All'   ,np.NaN,np.NaN,np.NaN,np.NaN],
    ['Custom',np.NaN,np.NaN,np.NaN,np.NaN],
], columns=('PRESET','XMIN','XMAX','YMIN','YMAX'))
dfZoomPresets

def update_plot_mod_vs_obs(modeloption, trippurps, tlftypes, zoompreset, xmin, xmax, ymin, ymax):

    data = []
    for modelspace in modeloption:
        for trippurp in trippurps:
            for tlftype in tlftypes:
            
                # data for plotting from filtered dataframe
                plotdata1 = dfObsModelMerge[(dfObsModelMerge['TRIPPURP']==trippurp) & (dfObsModelMerge['TLFTYPE']==tlftype)]
                plotdata = plotdata1[(plotdata1['FF_CALIB_ROUND']==modelspace)]

                # fill any NaN values with zeros
                plotdata = plotdata.fillna(0)

                xplot = plotdata['BIN']
                yplot = plotdata['FREQ']
                name  = trippurp + ' ' + tlftype + ' (' + modelspace + ')'

                trace1 = go.Scatter(
                    x=xplot,
                    y=yplot,
                    mode='lines',
                    name=name,
                    line=dict(
                        shape='spline'
                    )
                )
                data.append(trace1)
                
    # get zoom presets
    if (zoompreset=='Custom'):
        _xmin = xmin
        _xmax = xmax
        _ymin = ymin
        _ymax = ymax
    else:
        _xmin = dfZoomPresets[dfZoomPresets['PRESET']==zoompreset]['XMIN'].values[0]
        _xmax = dfZoomPresets[dfZoomPresets['PRESET']==zoompreset]['XMAX'].values[0]
        _ymin = dfZoomPresets[dfZoomPresets['PRESET']==zoompreset]['YMIN'].values[0]
        _ymax = dfZoomPresets[dfZoomPresets['PRESET']==zoompreset]['YMAX'].values[0]

    layout = go.Layout(
        title='Trip Length Frequencies - Observed Data',
        yaxis=dict(
            title='',
            range=(_ymin,_ymax)
        ),
        xaxis=dict(
            title='GC/Dist/Time',
            range=(_xmin,_xmax)
        ),
        #width=1600,
        height=450
    )
    
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)

In [24]:
# MAKE INTERACTIVE CHART
py.offline.init_notebook_mode(connected=True)

lstTripPurp  = dfObsModelMerge['TRIPPURP'].unique().tolist() # HBW, HBShp, HBOth,...
lstTLFType   = dfObsModelMerge['TLFTYPE' ].unique().tolist() # GC, Time, Dist
lstModOpt    = dfObsModelMerge['FF_CALIB_ROUND'].unique().tolist()
    
selectTripPurp     = widgets.SelectMultiple(options=lstTripPurp                     , value=('HBW',)               , description='Trip Purpose')
selectTLFType      = widgets.SelectMultiple(options=lstTLFType                      , value=('GC' ,)               , description='TLF Type'    )
selectPreset       = widgets.Select        (options=dfZoomPresets['PRESET'].tolist(), value= 'All'                 , description='Zoom Presets')
selectModelOption  = widgets.SelectMultiple(options=lstModOpt                       , value=('0-Initial from v832',), description='Model Run')

#custom extents
xmin = widgets.Text(value='0'  , description='X Min')
xmax = widgets.Text(value='60' , description='X Max')
ymin = widgets.Text(value='0'  , description='Y Min')
ymax = widgets.Text(value='.01', description='Y Max')

widgets.interactive(update_plot_mod_vs_obs, modeloption=selectModelOption,trippurps=selectTripPurp, tlftypes=selectTLFType, zoompreset=selectPreset, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)

interactive(children=(SelectMultiple(description='Model Run', index=(0,), options=('0-Initial from v832', '1-S…

# Friction Factor Adjustment

In [31]:
dfsTLF_TDM[0]

,TLFTYPE,TRIPPURP,BIN,FREQ
0,Dist,Ext_HV,0,5.526880e-04
1,Dist,Ext_HV,2,4.525133e-03
2,Dist,Ext_HV,4,1.253974e-02
3,Dist,Ext_HV,6,2.535801e-02
4,Dist,Ext_HV,8,2.802756e-02
...,...,...,...,...
7408,Time,XX,192,1.421322e-04
7409,Time,XX,194,3.194715e-05
7410,Time,XX,196,6.519827e-07
7411,Time,XX,198,1.303965e-05


In [41]:
dfsFF[0]

,BIN,TRIPPURP,FF
0,0,HBW,1.000000
1,2,HBW,0.999800
2,4,HBW,0.609169
3,6,HBW,0.190323
4,8,HBW,0.098000
...,...,...,...
146,292,XI_HV,0.000000
147,294,XI_HV,0.000000
148,296,XI_HV,0.000000
149,298,XI_HV,0.000000


In [50]:
# calculated new friction factors for each round of calibration

for index, row in dfCalibrationRounds.iterrows():
    calib_round = row['FF_CALIB_ROUND']
    print('Calbration Round: ' + calib_round)

    # if friction factors not locked
    if row['LOCKFF']==False:

        # initialize dataframes with round data, round index should be same as indexes for TLF and FF
        dfTLF_TDM = dfsTLF_TDM[index][dfsTLF_TDM[index]['TLFTYPE']=='GC'].copy()
        dfFF      = dfsFF[index].copy()
        
        # calculate adjustment factor
        dfTLF_TDM.rename(columns={'FREQ':'FREQ_TDM'},inplace=True)
        dfTLF_Obs.rename(columns={'FREQ':'FREQ_Obs'},inplace=True)

        dfModVsObs = pd.DataFrame.merge(dfTLF_TDM,dfTLF_Obs,on=('TRIPPURP','TLFTYPE','BIN'))

        # adjustment factor is observed divided by model
        dfModVsObs['ADJFACTOR'] = dfModVsObs['FREQ_Obs'] / dfModVsObs['FREQ_TDM']

        # fill NaNs with zeros
        dfModVsObs.fillna(0,inplace=True)
                
        if debug: display(dfModVsObs)

        # raw adjusted friction factor: multiply friction factor from previous round by adjustment factor
        dfModVsObsWithFF = pd.DataFrame.merge(dfModVsObs,dfFF,on=('TRIPPURP','BIN'))
        if debug: display(dfModVsObsWithFF)

    # if friction factors locked
    else:

        df = pd.read_csv("temp")




# normalized adj friction factor: divide by max (round to seven decimals)

# manual adjustments

# new friction factors for next round

Calbration Round: 0-Initial from v832


,TLFTYPE_x,TRIPPURP,BIN,FREQ_TDM,FREQ_Obs,TLFTYPE_y,ADJFACTOR
0,GC,HBOth,0,0.000000,0.466045,Dist,inf
1,GC,HBOth,0,0.000000,0.000000,Time,0.000000
2,GC,HBOth,0,0.000000,0.000000,GC,0.000000
3,GC,HBOth,2,0.086122,0.217005,Dist,2.519744
4,GC,HBOth,2,0.086122,0.095379,Time,1.107482
...,...,...,...,...,...,...,...
2480,GC,XI,292,0.000000,0.000143,GC,inf
2481,GC,XI,294,0.000000,0.000139,GC,inf
2482,GC,XI,296,0.000000,0.000136,GC,inf
2483,GC,XI,298,0.000000,0.000132,GC,inf


KeyError: 'TLFTYPE'